In [ ]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
trained_model = "./gemma3-phase1-best"
tokenizer = AutoTokenizer.from_pretrained(trained_model)
model = AutoModelForCausalLM.from_pretrained(trained_model)
model.eval()

In [ ]:
# ====== Load dataset ======
def load_partition(path: str) -> Dataset:
    df = pd.read_csv(path).head(10)
    return Dataset.from_pandas(df)

dataset = load_partition("./Student_Training_Data/GPT.csv")

print(f"Loaded {len(dataset)} samples from dataset.")

In [ ]:
import json

predictions = []

for example in dataset:
    prompt = (
        "Classify the following scientific text as one of [background, method, result].\n\n"
        f"Text: {example['string']}\n"
        "Provide your classification."
    )

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            num_beams=1
        )

    generated = tokenizer.decode(output[0], skip_special_tokens=True)

    classification = "parse_error"
    for label in ["background", "method", "result"]:
        if label in generated.lower():
            classification = label
            break

    predictions.append({
        "predicted_classification": classification,
        "predicted_reasoning": "None"
    })
    print(f"Predicted: {classification} Generated: {generated} ")

# ====== Save to CSV ======
df = dataset.to_pandas()
df["predicted_classification"] = [p["predicted_classification"] for p in predictions]
df["predicted_reasoning"] = [p["predicted_reasoning"] for p in predictions]
df.to_csv("gemma3_classification_results.csv", index=False)

In [ ]:
# Perform inference step here
def generate_output(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

predictions = []

for index, row in dataset.iterrows():
    text_input = row["string"]
    predicted_label = generate_output(text_input)
    predictions.append(predicted_label)
    print("classification: ", predicted_label)

# Save to DataFrame
dataset["predicted_classification"] = predictions
dataset.to_csv("gemma3_classification_results.csv", index=False)
print("Inference for first 10 data points completed.")
# Save results
# dataset = dataset.add_column("generated_reasoning", predictions)
# output_csv_path = "./llama-student-phase2.csv"
# dataset.to_pandas().to_csv(output_csv_path, index=False)

# print(f"Inference completed. Results saved to {output_csv_path}")

In [ ]:
for i, prediction in enumerate(predictions):
    print(f"Index {i}: {prediction} \n")